<a href="https://colab.research.google.com/github/EarthChart0809/Programming-subject-4/blob/main/subject_04_10_KR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **プログラムの名称**

## **概要**
このプログラムは、簡単なテトリスゲームを実装しています。 \
主な機能として、TetrisBlock クラスでランダムなブロックを生成し、TetrisCanvas クラスでフィールドとブロックを描画します。TetrisField クラスでゲームフィールドを管理し、行が揃ったときに削除・移動します。move_block メソッドでブロックを移動させ、衝突を判定し、衝突があればブロックを固定して新しいブロックを生成します。auto_drop 関数でブロックが自動的に下に落ち、ボタンで手動操作も可能です。ipywidgets で操作ボタンを表示し、pygame でゲーム状態を画像表示します。そして、難易度変更機能とスコアボード機能も取り付け、ブロックを固定したときに20ポイント、行の削除をした際100ポイントを得られるようにしました。\
これらの機能を組み合わせることで、基本的なテトリスゲームを実現しています。


## **操作方法**
pythonの標準機能であるipywidgets を使って、テトリスゲームの操作ボタン(←,→,↓,⟳)を表示し、ユーザーがこれらのボタンを使ってゲームを操作できるようにします。

## **制作者のコメント**
このプログラムを作成するときに頑張った点は「Easy,Normal,Hard」の中からゲーム難易度を選ぶことができるようにしたことです。
「Hard」は自動落下の仕方と操作方法の癖が強くかなり難しくなってしまいましたが慣れれば楽しむことができると思います。また、Googlecolabという制約の中で自分が思い描いていたプログラムに近づけることができたので良かったです。

このブロックが欲しいとなった場合は、座標を用いたブロックの作成をしてオリジナルのブロックを作り上げることも可能です。

高得点を目指してぜひ取り組んでみてください。


## **作成時間**

作成時間：**約26時間**


In [ ]:
#必ず実行してください
%reset -f

In [ ]:
#テトリスゲームの実行に必要なpip
!pip install ipycanvas
!pip install pygame
!pip install pillow

In [1]:
#実行するとテトリスゲームがスタートします
import pygame
from google.colab import output
import numpy as np
import time
import io
from PIL import Image
from IPython.display import display, clear_output
from ipywidgets import Button, HBox, VBox, Output,Dropdown,Layout
import random
import threading

# 定数
BLOCK_SIZE = 25  # ブロックの縦横サイズpx
FIELD_WIDTH = 10  # フィールドの幅
FIELD_HEIGHT = 20  # フィールドの高さ

MOVE_LEFT = 0  # 左にブロックを移動することを示す定数
MOVE_RIGHT = 1  # 右にブロックを移動することを示す定数
MOVE_DOWN = 2  # 下にブロックを移動することを示す定数

# ブロックを構成する正方形のクラス
class TetrisSquare():
    def __init__(self, x=0, y=0, color="gray"):
        self.x = x
        self.y = y
        self.color = color

    def set_cord(self, x, y):
        self.x = x
        self.y = y

    def get_cord(self):
        return int(self.x), int(self.y)

    def set_color(self, color):
        self.color = color

    def get_color(self):
        return self.color

    def get_moved_cord(self, direction):
        x, y = self.get_cord()
        if direction == MOVE_LEFT:
            return x - 1, y
        elif direction == MOVE_RIGHT:
            return x + 1, y
        elif direction == MOVE_DOWN:
            return x, y + 1
        else:
            return x, y

# テトリス画面を描画するキャンバスクラス
class TetrisCanvas():
    def __init__(self, field):
        self.width = field.get_width() * BLOCK_SIZE
        self.height = field.get_height() * BLOCK_SIZE
        self.screen = pygame.Surface((self.width, self.height))
        self.block_size = BLOCK_SIZE  # ブロックのサイズを設定

    def update(self, field, block):
        self.screen.fill((255, 255, 255))

        # マス目の描画
        for y in range(field.get_height()):
            for x in range(field.get_width()):
                pygame.draw.rect(
                    self.screen,
                    pygame.Color("black"),
                    pygame.Rect(x * self.block_size, y * self.block_size, self.block_size, self.block_size),
                    1  # 線の太さ
                )

        # フィールドのブロックを描画
        for y in range(field.get_height()):
            for x in range(field.get_width()):
                square = field.get_square(x, y)
                color = square.get_color()
                if color != "gray":
                    pygame.draw.rect(
                        self.screen,
                        pygame.Color(color),
                        pygame.Rect(x * self.block_size, y * self.block_size, self.block_size, self.block_size),
                        0
                    )

        # 移動中のブロックを描画
        if block:
            for square in block.get_squares():
                x, y = square.get_cord()
                color = square.get_color()
                pygame.draw.rect(
                    self.screen,
                    pygame.Color(color),
                    pygame.Rect(x * self.block_size, y * self.block_size, self.block_size, self.block_size),
                    0
                )

    def get_screen(self):
        return self.screen

# 積まれたブロックの情報を管理するフィールドクラス
class TetrisField():
    def __init__(self):
        self.width = FIELD_WIDTH
        self.height = FIELD_HEIGHT
        self.squares = []
        for y in range(self.height):
            for x in range(self.width):
                self.squares.append(TetrisSquare(x, y, "gray"))

    def get_width(self):
        return self.width

    def get_height(self):
        return self.height

    def get_squares(self):
        return self.squares

    def get_square(self, x, y):
        return self.squares[y * self.width + x]

    def judge_game_over(self, block):
        no_empty_cord = set(square.get_cord() for square in self.get_squares() if square.get_color() != "gray")
        block_cord = set(square.get_cord() for square in block.get_squares())
        collision_set = no_empty_cord & block_cord
        return len(collision_set) > 0

    def judge_can_move(self, block, direction):
        no_empty_cord = set(square.get_cord() for square in self.get_squares() if square.get_color() != "gray")
        move_block_cord = set(square.get_moved_cord(direction) for square in block.get_squares())
        for x, y in move_block_cord:
            if x < 0 or x >= self.width or y < 0 or y >= self.height:
                return False
        collision_set = no_empty_cord & move_block_cord
        return len(collision_set) == 0

    def fix_block(self, block):
        for square in block.get_squares():
            x, y = square.get_cord()
            color = square.get_color()
            field_square = self.get_square(x, y)
            field_square.set_color(color)

    def delete_line(self):
        lines_deleted = 0
        for y in range(self.height):
            for x in range(self.width):
                square = self.get_square(x, y)
                if square.get_color() == "gray":
                    break
            else:
                lines_deleted += 1
                for down_y in range(y, 0, -1):
                    for x in range(self.width):
                        src_square = self.get_square(x, down_y - 1)
                        dst_square = self.get_square(x, down_y)
                        dst_square.set_color(src_square.get_color())
                for x in range(self.width):
                    square = self.get_square(x, 0)
                    square.set_color("gray")
        return lines_deleted

    def judge_can_rotate(self, block, new_cords):
        no_empty_cord = set(square.get_cord() for square in self.get_squares() if square.get_color() != "gray")
        for x, y in new_cords:
            if x < 0 or x >= self.width or y < 0 or y >= self.height:
                return False
        collision_set = no_empty_cord & set(tuple(cord) for cord in new_cords)
        return len(collision_set) == 0

    def delete_full_lines(self):
        lines_deleted = 0  # 消去されたライン数のカウント
        # フィールドの高さを逆順でループする
        for y in range(self.height - 1, -1, -1):
            # その行がすべて埋まっているか確認
            if all(self.get_square(x, y).get_color() != "gray" for x in range(self.width)):
                # 埋まっている場合、その行を削除して、上の行を一つ下に移動
                for move_y in range(y, 0, -1):
                    for x in range(self.width):
                        self.get_square(x, move_y).set_color(self.get_square(x, move_y - 1).get_color())
                # 一番上の行を空にする
                for x in range(self.width):
                    self.get_square(x, 0).set_color("gray")
                # 行が削除されたので、再度同じy座標でチェック
                y += 1
        return lines_deleted  # 消去されたライン数を返す

# テトリスのブロックのクラス
class TetrisBlock():
    def __init__(self):
        self.squares = []
        #下記のコードからオリジナルのブロックを作成できます
        block_type = random.randint(1, 6) #ブロックの追加に合わせて数値を変更してください
        if block_type == 1:  # I字型ブロック
            color = "red"
            cords = [
                [FIELD_WIDTH // 2, 0],
                [FIELD_WIDTH // 2, 1],
                [FIELD_WIDTH // 2, 2],
                [FIELD_WIDTH // 2, 3],
            ]
        elif block_type == 2:  # O字型ブロック
            color = "blue"
            cords = [
                [FIELD_WIDTH // 2, 0],
                [FIELD_WIDTH // 2, 1],
                [FIELD_WIDTH // 2 - 1, 0],
                [FIELD_WIDTH // 2 - 1, 1],
            ]
        elif block_type == 3:  # 逆L字型ブロック
            color = "green"
            cords = [
                [FIELD_WIDTH // 2 - 1, 0],
                [FIELD_WIDTH // 2, 0],
                [FIELD_WIDTH // 2, 1],
                [FIELD_WIDTH // 2, 2],
            ]
        elif block_type == 4:  # L字型ブロック
            color = "orange"
            cords = [
                [FIELD_WIDTH // 2, 0],
                [FIELD_WIDTH // 2 - 1, 0],
                [FIELD_WIDTH // 2 - 1, 1],
                [FIELD_WIDTH // 2 - 1, 2],
            ]
        elif block_type == 5:  # T字型ブロック
            color = "purple"
            cords = [
                [FIELD_WIDTH // 2, 1],  # 中央
                [FIELD_WIDTH // 2 - 1, 1],  # 左
                [FIELD_WIDTH // 2 + 1, 1],  # 右
                [FIELD_WIDTH // 2, 0],  # 上
            ]
        elif block_type == 6:  # 蛇型ブロック
            color = "yellow"
            cords = [
                [FIELD_WIDTH // 2, 1],
                [FIELD_WIDTH // 2 - 1, 1],
                [FIELD_WIDTH // 2 + 1, 0],
                [FIELD_WIDTH // 2 ,0],
            ]
        for cord in cords:
            self.squares.append(TetrisSquare(cord[0], cord[1], color))

    def get_squares(self):
        return self.squares

    def move(self, direction):
        for square in self.squares:
            x, y = square.get_moved_cord(direction)
            square.set_cord(x, y)

    def rotate(self):
        center_x, center_y = self.squares[0].get_cord()  # 回転の中心
        new_cords = []
        for square in self.squares:
            x, y = square.get_cord()
            new_x = center_y - y + center_x
            new_y = x - center_x + center_y
            new_cords.append([new_x, new_y])
        return new_cords

    def set_cords(self, new_cords):
        for i, cord in enumerate(new_cords):
            self.squares[i].set_cord(cord[0], cord[1])

# テトリスゲームを制御するクラス
class TetrisGame():
    def __init__(self):
        self.field = TetrisField()
        self.block = None
        self.canvas = TetrisCanvas(self.field)
        self.canvas.update(self.field, self.block)
        self.difficulty = 1.0  # デフォルトの難易度（自動落下速度）
        self.score = 0  # 初期スコア

    def start(self):
        self.field = TetrisField()
        self.new_block()

    def new_block(self):
        self.block = TetrisBlock()
        if self.field.judge_game_over(self.block):
            print("GAMEOVER")
            return False
        self.canvas.update(self.field, self.block)
        return True

    def move_block(self, direction):
        if self.field.judge_can_move(self.block, direction):
            self.block.move(direction)
            self.canvas.update(self.field, self.block)
        else:
            if direction == MOVE_DOWN:
                self.fix_block()  # 修正後の fix_block メソッドを呼び出す
                if not self.new_block():
                    return False
        return True


    def rotate_block(self):
        new_cords = self.block.rotate()
        if self.field.judge_can_rotate(self.block, new_cords):
            self.block.set_cords(new_cords)
            self.canvas.update(self.field, self.block)


    def set_difficulty(self, level):
        self.difficulty = level

    def add_score(self, points):
        self.score += points
        print(f"Added {points} points. New Score: {self.score}")  # スコアの追加を表示

    def fix_block(self):
        self.field.fix_block(self.block)
        self.add_score(10)  # ブロックを固定するたびに10点加算
        lines_cleared = self.field.delete_full_lines()  # ライン消去処理を行う
        if lines_cleared > 0:
            self.add_score(100 * lines_cleared)  # ライン消去ごとにボーナススコア
        self.update_display()

    def update_display(self):
        self.canvas.update(self.field, self.block)
        with output_display:
            clear_output(wait=True)
            print(f"Score: {self.score}")
            display(Image.open(bio))

# イベントを受け付けてそのイベントに応じてテトリスを制御するクラス
class EventHandler():
    def __init__(self, game):
        self.game = game

    def start_event(self):
        self.game.start()

    def handle_events(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                return False
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    self.game.move_block(MOVE_LEFT)
                elif event.key == pygame.K_RIGHT:
                    self.game.move_block(MOVE_RIGHT)
                elif event.key == pygame.K_DOWN:
                    if not self.game.move_block(MOVE_DOWN):
                        return False
        return True

# Pygame初期化
pygame.init()

# ゲーム開始
game = TetrisGame()
event_handler = EventHandler(game)

# ディスプレイの表示
output_display = Output()

def update_display():
    screen = game.canvas.get_screen()
    img_str = pygame.image.tostring(screen, 'RGB')
    img = Image.frombytes('RGB', screen.get_size(), img_str)
    bio = io.BytesIO()
    img.save(bio, format='PNG')
    bio.seek(0)
    with output_display:
        clear_output(wait=True)
        print(f"Score: {game.score}")
        display(Image.open(bio))

def move_left(b):
    game.move_block(MOVE_LEFT)
    update_display()

def move_right(b):
    game.move_block(MOVE_RIGHT)
    update_display()

def move_down(b):
    game.move_block(MOVE_DOWN)
    update_display()

def rotate(b):
    game.rotate_block()
    update_display()

def auto_drop():
    while True:
        time.sleep(game.difficulty)
        if not game.move_block(MOVE_DOWN):
            break
        update_display()

# 難易度を選択するドロップダウンメニューを作成
difficulty_dropdown = Dropdown(
    options=[('Easy', 1.0), ('Normal', 0.75), ('Hard', 0.4)],
    value=1.0,
    description='難易度:',
    layout=Layout(width='150px')  # ドロップダウンの幅を設定
)

# 難易度が変更されたときの処理を定義
def on_difficulty_change(change):
    game.set_difficulty(change['new'])

# 難易度変更のイベントハンドラを設定
difficulty_dropdown.observe(on_difficulty_change, names='value')

# 各ボタンのレイアウトをコンパクトに設定
button_layout = Layout(width='70px', height='30px', margin='0px 5px 0px 5px')

# ボタンを作成
left_button = Button(description="←", layout=button_layout)
right_button = Button(description="→", layout=button_layout)
down_button = Button(description="↓", layout=button_layout)
rotate_botton = Button(description="⟳", layout=button_layout)

# ボタンのクリックイベントハンドラを設定
left_button.on_click(move_left)
right_button.on_click(move_right)
down_button.on_click(move_down)
rotate_botton.on_click(rotate)

game.start()
update_display()

# 自動ドロップを別スレッドで実行
threading.Thread(target=auto_drop, daemon=True).start()

# ボタンと表示を並べて表示
display(VBox([output_display, HBox([difficulty_dropdown,left_button, right_button, down_button, rotate_botton], layout=Layout(justify_content='center'))]))


pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
